In [1]:
from data_tools import get_data_with_dates, get_all_data
from tqdm.notebook import tqdm
from warnings import warn
import pandas as pd
from random import sample

In [2]:
data = get_data_with_dates(get_all_data())

In [3]:
samples = {}

for row in tqdm(data.itertuples(), total=len(data)):
    df = pd.read_excel(row.path)
    try:
        df = df[(df['Data wydania'] >= row.crisis_start - pd.Timedelta(days=30)) & (df['Data wydania'] < row.crisis_start + pd.Timedelta(days=30))]
        if df['Kryzys'].hasnans:
            if df['Kryzys'].nunique(dropna=False) != 2:
                warn(f'Invalid Kryzys column values in {row.name}.')
                continue
            df['label'] = ~df['Kryzys'].isna()
        else:
            df['Kryzys'] = df['Kryzys'].apply(lambda x: x[:3])
            if df['Kryzys'].nunique(dropna=False) != 2:
                warn(f'Invalid Kryzys column values in {row.name}.')
                continue
            df['label'] = df['Kryzys'] != 'NIE'
        sample_size = min(sum(df['label']), sum(~df['label']), 50)
        samples_neg = sample(df[~df['label']].index.to_list(), sample_size)
        samples_pos = sample(df[df['label']].index.to_list(), sample_size)
        samples[row.name] = samples_neg + samples_pos
        samples[row.name].sort()
    except KeyError:
        warn(f'{row.name} has shit keys.')

  0%|          | 0/91 [00:00<?, ?it/s]

nagranie Anny Lewandowskiej - nagranie z tańcem w stroju otyłej kobiety / konflikt z Mają Staśko has shit keys.


/tmp/ipykernel_272/1248414445.py:15: UserWarning: Invalid Kryzys column values in Ministerstwo Zdrowia - respiratory od handlarza bronią (baza wygenerowana wyłącznie dla przekazu kryzysowego; całość - 6,5 mln publikacji).
  warn(f'Invalid Kryzys column values in {row.name}.')


In [5]:
for key in samples:
    print(len(samples[key]))

100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
90
100
100
24
100
100
34
100
100
36
52
100
100
16
100
20
100
70
40
100
100
100
48
64
100
4
100
100
100
100
10
92
98
78
100
70
50
32
36
14
100
100
100
100
90
72
100
100
100
100
100
100
100
100
100
100
100
100


In [27]:
import json

with open('samples.json', 'w') as f:
    json.dump(samples, f)

In [26]:
n = 35
df = pd.read_excel(data['path'][n])
df.iloc[samples[data['name'][n]]]['Kryzys'].value_counts()

Kryzys
NIE                                              12
TAK - sprawa zatrudnienia kobiety w wieku 40+    12
Name: count, dtype: int64